# Structural Design Patterns

- Previously, we covered possible strategies to simplify/beautify object creation in cases where the usual way is too tedious/untidy/ugly

- Now, we look at ways to assemble objects which allows for more flexible and extensible code, like Lego

- Structural design patterns explain how to assemble objects and classes into larger structures, while keeping these structures flexible and efficient.

## 1. Adapter

### Big Picture

- Literally what the name suggests. 

- Imagine you have some unmodifiable client code, and some unmodifiable server code. The client's expected input and the server's output do not match

- So instead of doing a rewrite of your client, you simply write a class dedicated to changing your server's output into something that your client can accept

### Example

#### Via Inheritance

In [8]:
class Target:
    def request(self) -> str:
        return "Target: The default target's behavior."


class Adaptee:
    def adaptee_request(self) -> str:
        return ".eetpadA eht fo roivaheb laicepS"

class Adapter(Target, Adaptee):
    ## Overwriting the parent method "Target.request", to act on "Adaptee.specific_request"
    def request(self) -> str:
        return f"Adapter: (TRANSLATED) {self.adaptee_request()[::-1]}"

def client_code(target: Target) -> None:
    print(target.request(), end="")

## Client code works fine with the `Target` object
target = Target()
print(client_code(target))

## But it doesn't understand the `Adaptee` class's output
adaptee = Adaptee()
try:
    client_code(adaptee)
except:
    print('client cannot work with adaptee class')

## Use adaptor to give the adaptee code an interface that client can work with
adapter = Adapter()
print(client_code(adapter))

Target: The default target's behavior.None
client cannot work with adaptee class
Adapter: (TRANSLATED) Special behavior of the Adaptee.None


#### Via Object Composition

In [11]:
class Target:
    def request(self) -> str:
        return "Target: The default target's behavior."


class Adaptee:
    def adaptee_request(self) -> str:
        return ".eetpadA eht fo roivaheb laicepS"

class Adapter(Target):
    def __init__(self, adaptee: Adaptee) -> None:
        self.adaptee = adaptee

    ## Overwrite `Target`'s request method
    def request(self) -> str:
        return f"Adapter: (TRANSLATED) {self.adaptee.adaptee_request()[::-1]}"


def client_code(target: Target) -> None:
    print(target.request(), end="")


target = Target()
print(client_code(target))

try:
    adaptee = Adaptee()
    client_code(adaptee)
except:
    print('cannot work with adaptee class')

adapter = Adapter(adaptee)
client_code(adapter)

Target: The default target's behavior.None
cannot work with adaptee class
Adapter: (TRANSLATED) Special behavior of the Adaptee.

### Discussion

- Your new Adaptor can be implemented in one of two ways
    1. Your adaptor implements the interface of the client, and wraps around the service object.
        - So instead of calling the client with the incompatible input, you call the adaptor with the SAME METHOD NAMES as if you were calling the client
        - Since this works via interface, no coupling between client and adaptor classes are introduced!
    2. Your adaptor can inherit from both the client and server classes
        - If you do it this way, you override the relevant methods from the client

### Pros and Cons

- Pros:
    - Single Responsibility Principle: You can separate the interface or data conversion code from the primary business logic of the program.
    - Open/Closed Principle. You can always add new adapters without breaking the client code, so long as they respect the client interface

- Cons:
    - More complexity

### Use Cases

- You want to use some existing class, but its interface isn’t compatible with the rest of your code

- Use the pattern when you want to reuse several existing subclasses that lack some common functionality that can’t be added to the superclass.

## 2. Bridge

### Big Picture

- Imagine you are dealing with a huge class that handles 2 orthogonal concepts
    - e.g. If you have a `Geometry` class, each object can have a `Shape` dimension, and a `Colour` dimension

- If you try to develop everything under the `Geometry` class, you can end up with a big mess, because your methods deal with 2 quite distinct concepts

- As such, it can be cleaner to split these into 2 different sets of objects, and use composition to have one rely on another

- i.e. the remote control

### Example

In [31]:
class Colour:
    def __init__(self, colour):
        self._colour = colour
    
    def fill_shape(self, shape: 'Shape'):
        print(f"Displaying {shape} with {self._colour}")

class Shape:
    def __init__(self, shape: str, colour: Colour):
        self._shape: str = shape
        self._colour: Colour = colour

    def __repr__(self):
        return self._shape

    def show_shape_skeleton(self):
        print(f'Displaying {self._shape}')

    def show_shape_with_colour(self):
        # self.show_shape_skeleton()
        self._colour.fill_shape(self)

blue_colour = Colour('blue')
circle = Shape('circle', blue_colour)
circle.show_shape_skeleton()
circle.show_shape_with_colour()

Displaying circle
Displaying circle with blue


### Discussion

- In the example above, the `Shape` object (the abstraction) wants to display itself with colour, but delegates the job of doing that to the `Colour` class (the implementation)
- As stated above, the objective here is to allow Shape to be extended easily without affecting how it is coloured
    - For example, you may wish to create an extension that involves making a `3DShape` class

- Think of `Colour` as a remote control, for `Shape` to achieve an outcome (colour the shape) without knowing how the outcome is achieved

### Pros and Cons

- Pros
    - You can create platform-independent classes and apps
        - By separating out the implementation into orthogonal classes, you are have more freedom to mix and match components
    - The client code works with high-level abstractions. It isn’t exposed to the platform details.
    - Open/Closed Principle. You can introduce new abstractions and implementations independently from each other.
    - Single Responsibility Principle. You can focus on high-level logic in the abstraction and on platform details in the implementation.

- Cons
    - Can make code unnecessarily complicated. If your class is small, it may be good to tolerate a bit of bad practise to keep everything within a single class

### Use Cases

- Use the Bridge pattern when you want to divide and organize a monolithic class that has several variants of some functionality
    - That way, you help to separate the concerns of the relevant classes into individual hierachies

- Use the pattern when you need to extend a class in several orthogonal (independent) dimensions.

## 3. Composite

### Big Picture

- There are occasions when you have objects nested within objects nested within objects
- This sort of complicated object can be a huge bitch to write code for, because you basically have to "unwrap" the entire chain before you can perform any operations

- Let's take an example:
    - Suppose you run an e-commerce marketplace
    - You have many stores onboarded, and each store may have several sub-brands and/or substores
    - A user should be able to "add to cart" from any of these stores/substores
    - When a user views their cart, they should be able to see the aggregate amount they are spending on each store/substore, as well as the total amount they spend overall

    - In a typical usage pattern, you may simply write a `Cart` object that sums the cost of all objects within it
        - But a cart can contain individual objects, or a collection of objects from  `Store`, or `Substore`
        - So to get the total sum, the `Cart` needs to visit every collection and request the value of the store/substore
    
- In this example, if your object collections follow a different interface from `Cart`, you're in trouble, because you need to perform the correct aggregation at each step without knowing how to get each object to compute the total sum of values

### Example

In [64]:
from abc import ABC, abstractmethod
from typing import Optional


class Component(ABC):

    def __init__(self):
        self._parent = None

    @property
    def parent(self) -> Optional['Component']:
        return self._parent

    @parent.setter
    def parent(self, parent: Optional['Component']):
        self._parent = parent

    @abstractmethod
    def operation(self) -> str:
        pass

    def add(self, component: 'Component') -> None:
        pass

    def remove(self, component: 'Component') -> None:
        pass

    def is_composite(self) -> bool:
        return False


class Leaf(Component):
    def __init__(self):
        self._parent: Optional[Component] = None

    @property
    def parent(self) -> Optional[Component]:
        return self._parent
    
    @parent.setter
    def parent(self, parent: Optional[Component]) -> None:
        self._parent: Optional[Component] = parent
    
    def operation(self) -> str:
        return "Leaf"
    
class Composite(Component):

    def __init__(self) -> None:
        self._children: list[Component] = []
    
    def add(self, component: Component) -> None:
        self._children.append(component)
        component.parent = self

    def remove(self, component: Component) -> None:
        self._children.remove(component)
        component.parent = None

    def is_composite(self) -> bool:
        return True

    def operation(self) -> str:
        results: list[str] = []
        for child in self._children:
            results.append(child.operation())
        return f"Branch({'+'.join(results)})"


def client_code(component: Component) -> None:
    print(f"RESULT: {component.operation()}", end="\n")


def client_code2(component1: Component, component2: Component) -> None:
    if component1.is_composite():
        component1.add(component2)

    print(f"RESULT: {component1.operation()}", end="\n")


leaf = Leaf() ## Leaf node
branch = Composite() ##branch with no children
client_code(leaf)
client_code(branch)

branch.add(Leaf())
branch.add(Leaf())
client_code(branch) ##Branch with children

branch2 = Composite()
branch2.add(Leaf())
branch.add(branch2)
client_code(branch) ##Branch with children and another branch

client_code2(branch, leaf) ##Client code can operate on composite object without having a full view of what's inside it

RESULT: Leaf
RESULT: Branch()
RESULT: Branch(Leaf+Leaf)
RESULT: Branch(Leaf+Leaf+Branch(Leaf))
RESULT: Branch(Leaf+Leaf+Branch(Leaf)+Leaf)


### Discussion

- Think of this as some sort of "recursion"
- But instead of having each request propagate through the tree during run time, each composite object stores its state, so you can request for it dynamically

### Pros and Cons

- Pros
    - With this approach, clients can work with Composite objects easily, offloading the work of understanding the structure to the object itself
    - Open/Closed Principle. You can introduce new element types into the app without breaking the existing code, which now works with the object tree.

- Cons
    - Having a common interface can be difficult if the composite classes differ strongly. Don't overgeneralise the interface; it can become very hard to understand

### Use Cases

- Use the Composite pattern when you have to implement a tree-like object structure.

- Use the pattern when you want the client code to treat both simple and complex elements the same way

## 4. Decorator

### Big Picture

- Python has a built-in idea of decorators, so this should not be entirely foreign
- The idea is similar to `Adapter`, where you create a new class to wrap around an existing class
    - In the case of Adapter, you do this to provide a different interface from accessing the object, or to modify the object's output in some way
    - In a Decorator, you 
- The point of an decorator is to let you grant new behaviour to an object simply by wrapping it with something

### Example

In [69]:
from typing import Optional
from abc import ABC, abstractmethod

class ComponentBase(ABC):
    @abstractmethod
    def operation(self) -> str:
        raise NotImplementedError()

class SimpleComponent(ComponentBase):
    def operation(self) -> str:
        return "SimpleComponent"

class DecoratorBase(ComponentBase):

    def __init__(self, component: ComponentBase) -> None:
        self._component: ComponentBase = component

    @property
    def component(self) -> ComponentBase:
        return self._component

    ## Applies the same interface
    @abstractmethod
    def operation(self) -> str:
        return self._component.operation()

class ConcreteDecoratorA(DecoratorBase):
    def operation(self) -> str:
        return f"ConcreteDecoratorA({self.component.operation()})"

class ConcreteDecoratorB(DecoratorBase):
    def operation(self) -> str:
        return f"ConcreteDecoratorB({self.component.operation()})"


def client_code(component: ComponentBase) -> None:
    print(f"RESULT: {component.operation()}", end="\n")

simple_component = SimpleComponent()
client_code(simple_component)

decorated_simple_component_a = ConcreteDecoratorA(simple_component)
client_code(decorated_simple_component_a)

decorated_simple_component_a_b = ConcreteDecoratorB(decorated_simple_component_a)
client_code(decorated_simple_component_a_b)

RESULT: SimpleComponent
RESULT: ConcreteDecoratorA(SimpleComponent)
RESULT: ConcreteDecoratorB(ConcreteDecoratorA(SimpleComponent))


### Discussion

- Generally, subclassing would be easier and cleaner whenever different behaviours are involved
- But this design can be useful if you have some minor differences in behaviour that don't necessarily require something as verbose as an entire subclass

### Pros and Cons

- Pros
    - Extend an object’s behavior without making a new subclass
    - Add/remove responsibilities from an object at runtime
    - You can combine several behaviors by wrapping an object into multiple decorators
    - Single Responsibility Principle: You can divide a monolithic class that implements many possible variants of behavior into several smaller classes.

- Cons
    - It’s hard to remove a specific wrapper from the wrappers stack
    - It’s hard to implement a decorator in such a way that its behavior doesn’t depend on the order in the decorators stack
    - The initial configuration code of layers might look pretty ugly

### Use Cases

- Use the Decorator pattern when you need to be able to assign extra behaviors to objects at runtime without breaking the code that uses these objects.

- Use the pattern when it’s awkward or not possible to extend an object’s behavior using inheritance.
    - e.g. Python class has `typing.Final` as type annotation, or `typing.final` as decorator, for linters to do static checks
    - e.g. Java has `final` keyword to block inheritance from a class


## 5. Facade

### Big Picture

- You have some code that makes use of a bunch of tools and libraries
- Though there is a lot happening "Under the hood", you shouldn't expect users to know what is happening
    - For example, XGBoost library is super complex under the hood, but you shouldn't expect users to call a bunch of methods (either those provided by you, or those in 3rd party libraries)
    - Users expect a simple interface. Fit and Predict.

- A Facade class does exactly this

### Example

In [70]:
from __future__ import annotations

class Facade:

    def __init__(self, subsystem1: Subsystem1, subsystem2: Subsystem2) -> None:
        self._subsystem1: Subsystem1 = subsystem1
        self._subsystem2: Subsystem2 = subsystem2

    def operation(self) -> str:
        results = []
        results.append("Facade initializes subsystems:")
        results.append(self._subsystem1.operation1())
        results.append(self._subsystem2.operation1())
        results.append("Facade orders subsystems to perform the actions in some order:")
        results.append(self._subsystem1.operation_n())
        results.append(self._subsystem2.operation_z())
        return "\n".join(results)

class Subsystem1:
    def operation1(self) -> str:
        return "Subsystem1: Ready!"

    def operation_n(self) -> str:
        return "Subsystem1: Go!"


class Subsystem2:
    def operation1(self) -> str:
        return "Subsystem2: Get ready!"

    def operation_z(self) -> str:
        return "Subsystem2: Fire!"

def client_code(facade: Facade) -> None:
    print(facade.operation(), end="\n")

subsystem1 = Subsystem1()
subsystem2 = Subsystem2()
facade = Facade(subsystem1, subsystem2)
client_code(facade)

Facade initializes subsystems:
Subsystem1: Ready!
Subsystem2: Get ready!
Facade orders subsystems to perform the actions in some order:
Subsystem1: Go!
Subsystem2: Fire!


### Discussion

- Write a class that wraps around your subsystems, in order to provide an easier interface to work with them
- Kind of like an Adapter, but an Adapter tends to only work with 1 class, while Facades can work with multiple
    - Adapter is literally just adapting the interfaces of mutually incompatible classes. The problem solved is interface compatibility.
    - Facades orchestrate behaviour. The problem solved is difficulty of users knowing what steps to take. 

### Pros and Cons

- Pros
    - You can isolate your code from the complexity of a subsystem

- Cons
    - If you overdo it, a facade becomes a huge "god object", that is coupled to all classes of your app. So anytime you change anything, you need to modify the Facade, which is super not ideal

### Use Cases

- Use the Facade pattern when you need to have a limited but straightforward interface to a complex subsystem.

- Use the Facade when you have multiple systems that need to talk to each other
    - Rather than leaving systems to communicate with each other via verbose and extended operations on both sides, create a Facade for each system to orchestrate the work, and simplify the communication between both systems


## 6. Flyweight

### Big Picture

- You have some object in your system where, ahead of time, you know that multiple instances will be created
- Though these objects are all slightly different, there is a large degree of overlap
- But if you create a new object each time, you will store a new copy of the repeated stuff for each distinct object
- This creates a huge memory overhead for no reason

- A flyweight allows you to support a huge number of these objects by sharing the repeated parts of the objects' states, and only varying the bits that are different

### Example

In [86]:
import json

## Create a "Flyweight", which you can think of as a "partial" class, awaiting the addition of some necessary attributes for it to be used
class Flyweight():
    def __init__(self, shared_state: str) -> None:
        self._shared_state: str = shared_state

    def operation(self, unique_state: list[str]) -> None:
        print(f"Flyweight: Displaying shared state: ({self._shared_state}) and unique state: ({'_'.join(unique_state)})", end="\n")

## Even with the flyweight class, you need a "manager" of sorts to monitor whether the specific flyweight has already been created
## If it has, we return it
## Else, we create it
class FlyweightFactory():
    
    _flyweights: dict[str, Flyweight] = {}

    def __init__(self, initial_flyweights: list[list[str]]) -> None:
        for state_list in initial_flyweights:
            key = self.concatenate_state_keys(state_list)
            self._flyweights[key] = Flyweight(key)

    def concatenate_state_keys(self, state: list[str]) -> str:
        return "_".join(state)

    def get_flyweight(self, shared_state: list[str]) -> Flyweight:
        key = self.concatenate_state_keys(shared_state)

        if not self._flyweights.get(key):
            print("FlyweightFactory: Can't find a flyweight, creating new one.")
            self._flyweights[key] = Flyweight(key)
        else:
            print("FlyweightFactory: Reusing existing flyweight.")

        return self._flyweights[key]

    def list_flyweights(self) -> None:
        count = len(self._flyweights)
        print(f"FlyweightFactory: I have {count} flyweights:")
        for key in self._flyweights.keys():
            print(f"    {key}", end='\n')

def add_car(factory: FlyweightFactory, plates: str, owner: str, brand: str, model: str, color: str) -> None:
    flyweight = factory.get_flyweight([brand, model, color])
    flyweight.operation([plates, owner])

factory = FlyweightFactory([
    ["Chevrolet", "Camaro2018", "pink"],
    ["Mercedes_Benz", "C300", "black"],
    ["Mercedes_Benz", "C500", "red"],
    ["BMW", "M5", "red"],
    ["BMW", "X6", "white"],
])

factory.list_flyweights()

add_car(
    factory, "CL234IR", "James Doe", "BMW", "M5", "red"
)

add_car(
    factory, "CL234IR", "James Doe", "BMW", "X1", "red"
)

factory.list_flyweights()

FlyweightFactory: I have 5 flyweights:
    Chevrolet_Camaro2018_pink
    Mercedes_Benz_C300_black
    Mercedes_Benz_C500_red
    BMW_M5_red
    BMW_X6_white
FlyweightFactory: Reusing existing flyweight.
Flyweight: Displaying shared state: (BMW_M5_red) and unique state: (CL234IR_James Doe)
FlyweightFactory: Can't find a flyweight, creating new one.
Flyweight: Displaying shared state: (BMW_X1_red) and unique state: (CL234IR_James Doe)
FlyweightFactory: I have 6 flyweights:
    Chevrolet_Camaro2018_pink
    Mercedes_Benz_C300_black
    Mercedes_Benz_C500_red
    BMW_M5_red
    BMW_X6_white
    BMW_X1_red


### Discussion

- The analogy here is, suppose you are coding an FPS game
- Each bullet you fire will have the same bullet animation. The only difference should be in trajectory etc
- So you don't really want to repeat the animation code for every bullet. This will clog up your RAM pretty quickly

### Pros and Cons

- Pros
    - You can save lots of RAM, assuming your program has tons of similar objects.

- Cons
    - If the "repeated stuff" requires computation each time it is called, you may be trading RAM for compute
    - The code becomes complicated, and you'll have to document clearly why this design was chosen

### Use Cases

- Use the Flyweight pattern only when your program must support a huge number of objects which barely fit into available RAM

## 7. Proxy

### Big Picture

- You have some underlying service object that clients want to access
- And you want to protect it, or monitor connections to it for whatever reason
- So you create a new object called a `Proxy` that clients interact with, that calls the service object on their behalf
    - You add the code you want to work with through this proxy

### Example

In [88]:
from abc import ABC, abstractmethod

class Subject(ABC):
    @abstractmethod
    def request(self) -> None:
        pass


class RealSubject(Subject):
    def request(self) -> None:
        print("RealSubject: Handling request.")

class Proxy(Subject):
    def __init__(self, real_subject: RealSubject) -> None:
        self._real_subject = real_subject

    def request(self) -> None:
        if self.check_access():
            self._real_subject.request()
            self.log_access()

    def check_access(self) -> bool:
        print("Proxy: Checking access prior to firing a real request.")
        return True

    def log_access(self) -> None:
        print("Proxy: Logging the time of request.", end="")

def client_code(subject: Subject) -> None:
    subject.request()

## Calling the real subject
real_subject = RealSubject()
client_code(real_subject)

print('='*50)

## To the client, calling the proxy object returns the same stuff, just with our added behaviours in the proxy class 
proxy = Proxy(real_subject)
client_code(proxy)

RealSubject: Handling request.
Proxy: Checking access prior to firing a real request.
RealSubject: Handling request.
Proxy: Logging the time of request.

### Discussion

- This is quite similar to `Decorator`
    - However, in `Decorator`, the client controls the initialisation of the Decarator class. Whereas in a `Proxy`, the client is unaware that it is not dealing with the underlying object

- This is quite similar to `Facade`, which also deals with a complex underlying entity and presents a simpler way of working with it
    - But in `Facade`, the point is to SIMPLIFY, so the Facade's interface is different from the underlying object
    - In `Proxy`, the interface is the same, so clients don't know what they are dealing with

### Pros and Cons

- Pros
    - You can control the service object without clients knowing about it.
    - You can manage the lifecycle of the service object when clients don’t care about it.
    - The proxy works even if the service object isn’t ready or is not available.
    - Open/Closed Principle. You can introduce new proxies without changing the service or clients.

- Cons
    - The code may become more complicated since you need to introduce a lot of new classes.
    - The response from the service might get delayed.

### Use Cases

- Lazy initialization (virtual proxy)
    - The underlying service object is huge and expensive to init (e.g. database connection), and it costs a lot of resources while it is up. 
    - So instead of creating the object the moment the app starts up, delay the start up until you need it

- Access control (protection proxy) 
    - Sometimes, you only want to allow specific clients/callers to access service object 
        - e.g. your object is some crucial component, and you want to protect it from malicious clients
    - So only pass it on after the client's credentials are checked

- Local execution of a remote service (remote proxy)
    - The service object is located on a remote server
    - The proxy handles the passing of the request over the network, rather than asking the client to handle the network details on its own

- Logging requests (logging proxy) 
    - You want to keep a history of requests to the service object
    - So you can get the proxy to log the requests somewhere, before passing the request on to the underlying objet

- Caching request results (caching proxy) 
    - Your service object can return really large results, and you want to cache them in the event of repeated queries
    - This is when you can cache results of client requests and manage the life cycle of this cache, especially if results are quite large.

- Smart reference
    - Suppose you want to know when no clients are using some heavy/expensive object, and you want to dismiss it once that happens (e.g. extended holidays etc)
    - You can use a proxy to check track live connections from clients, and if some conditions are met, remove the underlying service object